## Elastic Search HELM Installation How  To 

![tiny arrow](https://www.michael-wutzke.de/wp-content/uploads/2019/02/funktionsweise-elastic-stack-beats-logstash-elasticsearch-kibana.png)

In [2]:
kubectx 

tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
development


In [3]:
kubectx development

Switched to context "development".


In [4]:
kubens researchgroup1

Context "development" modified.
Active namespace is "researchgroup1".


## ADD THE HELM REPOSITORIES FOR ELATIC 

In [5]:
helm repo add elastic https://helm.elastic.co

"elastic" has been added to your repositories


## INSTALL ELASTIC SEARCH CLUSTER

Elasticsearch is a distributed, RESTful search and analytics engine capable of addressing a growing number of use cases. As the heart of the Elastic Stack, it centrally stores your data for lightning fast search, fine‑tuned relevancy, and powerful analytics that scale with ease.

![tiny arrow](https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/bltbc15ae12db3c3f87/5ea2b0f7e565460dbf2ce1eb/illustration-elasticheart-engine-blue-555x500px.png)


In [6]:
helm install elasticsearch elastic/elasticsearch 

NAME: elasticsearch
LAST DEPLOYED: Mon Jun 29 10:58:46 2020
NAMESPACE: researchgroup1
STATUS: deployed
REVISION: 1
NOTES:
1. Watch all cluster members come up.
  $ kubectl get pods --namespace=researchgroup1 -l app=elasticsearch-master -w
2. Test cluster health using Helm test.
  $ helm test elasticsearch --cleanup


In [12]:
kubectl get pods  -l app=elasticsearch-master 

NAME                     READY   STATUS    RESTARTS   AGE
elasticsearch-master-0   1/1     Running   0          101s
elasticsearch-master-1   1/1     Running   0          101s
elasticsearch-master-2   1/1     Running   0          101s


## CREATE A SERVICE FOR ELASTIC SEARCH 

In [13]:
cat << 'EOF' | kubectl apply -f - 
apiVersion: v1
kind: Service
metadata:
  name: elasticsearch
  labels:
    hpecp.hpe.com/hpecp-internal-gateway: "true"
spec:
  selector: 
    app: elasticsearch-master
  ports:
  - name: elastic
    protocol: TCP
    port: 9200
    targetPort: 9200 
  type: NodePort
EOF

service/elasticsearch created


## INSTALL KIBANA 

In [14]:
helm install kibana elastic/kibana 

NAME: kibana
LAST DEPLOYED: Mon Jun 29 11:04:44 2020
NAMESPACE: researchgroup1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [17]:
kubectl get pods -l app=kibana

NAME                             READY   STATUS              RESTARTS   AGE
kibana-kibana-6bcc4ddb7c-2zn8d   0/1     ContainerCreating   0          61s


## INSTALL THE KIBANA SERVICE 

Kibana is a free and open user interface that lets you visualize your Elasticsearch data and navigate the Elastic Stack. Do anything from tracking query load to understanding the way requests flow through your apps.

![tiny arrow](https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt14d57abad5d2499c/5ee0060cbf4ae52c761edcb7/illustrated-screenshot-kibana-7dot8-730x555.png)

In [30]:
## Set Service
cat << 'EOF' | kubectl create -f - 
apiVersion: v1
kind: Service
metadata:
  name: kibana
  labels:
    hpecp.hpe.com/hpecp-internal-gateway: "true"
spec:
  selector: 
    app: kibana
  ports:
  - name: http-kibana
    protocol: TCP
    port: 5601
    targetPort: 5601 
  type: NodePort
EOF

service/kibana created


In [32]:
kubectl describe svc kibana

Name:                     kibana
Namespace:                researchgroup1
Labels:                   hpecp.hpe.com/hpecp-internal-gateway=true
Annotations:              hpecp-internal-gateway/5601: ec2-15-236-36-5.eu-west-3.compute.amazonaws.com:10024
Selector:                 app=kibana
Type:                     NodePort
IP:                       10.96.2.162
Port:                     http-kibana  5601/TCP
TargetPort:               5601/TCP
NodePort:                 http-kibana  32554/TCP
Endpoints:                10.192.2.30:5601
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason  Age   From         Message
  ----    ------  ----  ----         -------
  Normal  HpeCp   99s   hpecp-agent  Created HPECP K8S service


In [36]:
kubectl describe svc elasticsearch-master 

Name:              elasticsearch-master
Namespace:         researchgroup1
Labels:            app=elasticsearch-master
                   app.kubernetes.io/managed-by=Helm
                   chart=elasticsearch
                   heritage=Helm
                   release=elasticsearch
Annotations:       meta.helm.sh/release-name: elasticsearch
                   meta.helm.sh/release-namespace: researchgroup1
Selector:          app=elasticsearch-master,chart=elasticsearch,release=elasticsearch
Type:              ClusterIP
IP:                10.96.89.70
Port:              http  9200/TCP
TargetPort:        9200/TCP
Endpoints:         10.192.1.230:9200,10.192.2.204:9200,10.192.2.29:9200
Port:              transport  9300/TCP
TargetPort:        9300/TCP
Endpoints:         10.192.1.230:9300,10.192.2.204:9300,10.192.2.29:9300
Session Affinity:  None
Events:            <none>


## Setup Kibana using  UI WITH THE INTERNAL DNS MASTER OF YOUR ES CLUSTER. 

Example:  elasticsearch-master.researchgroup1.svc.cluster.local:9300

`<ELASTICSEARCH SERVICE>.<NAMESPACE>.<SVC>.<YOUR-CLUSTER-DOMAIN>:<PORT>`

## INSTALL ELASTIC HQ AN OPENSOURCE CLUSTER MANAGER FOR ELASTIC SEARCH 

ElasticHQ is an open source application that offers a simplified interface for managing and monitoring Elasticsearch clusters.

![tiny arrow](https://www.elastichq.org/img/elastichq.png)

In [68]:
cat << 'EOF' | kubectl create -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: elastichq-app
spec:
  selector:
    matchLabels:
      app: elastichq-app
      tier: frontend
  template:
    metadata:
      labels:
        app:  elastichq-app
        tier: frontend
    spec:
     
      containers:
        - name: elastichq-app
          image: elastichq/elasticsearch-hq
          ports:
            - containerPort: 5000
              name: elastichq-port

---

apiVersion: v1
kind: Service
metadata:
  name: elastichq-app 
spec:
  selector: 
    app: elastichq-app
  ports:
  - name: http-elastichhq
    protocol: TCP
    port: 5000
    targetPort: 5000
  type: NodePort
          
          
EOF

deployment.apps/elastichq-app created
service/elastichq-app created


In [75]:
kubectl describe svc elastichq

Name:                     elastichq-app
Namespace:                researchgroup1
Labels:                   hpecp.hpe.com/hpecp-internal-gateway=true
Annotations:              hpecp-internal-gateway/5000: ec2-15-236-36-5.eu-west-3.compute.amazonaws.com:10028
Selector:                 app=elastichq-app
Type:                     NodePort
IP:                       10.96.67.124
Port:                     http-elastichhq  5000/TCP
TargetPort:               5000/TCP
NodePort:                 http-elastichhq  32710/TCP
Endpoints:                10.192.2.213:5000
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason  Age   From         Message
  ----    ------  ----  ----         -------
  Normal  HpeCp   27m   hpecp-agent  Created HPECP K8S service

